# English Parser

"BuT dAvId, ThAt's JuSt A mOnOiDaL-eNdOfUnCtOr-DeRiVeD cOnTeXt-CoErCiNg LaMbDa CoMbInAtOr In SiMpLeR tErMs"

In [5]:
# dependencies

import enum

from xnlp.parser import (
    Parser,
    runParser,
    satisfy,
    optional,
    zeroOrMore,
    oneOrMore
)

In this assignment, you are going to explore how to write parsing rules as described in class. The grammar you are going to design is a simplified version of English; words in such language consist of only 6 types:

- `ADJ`: Adjectives (e.g. `beautiful`, `big`)
- `ADV`: Adverbs (e.g. `gracefully`, `quickly`)
- `NOUN`: Noun (e.g. `penguin`, `penguin`, `penguin`)
- `PUNCT`: Punctuation (e.g. `,`, `;`, `.`)
- `VERB`: Verb (e.g. `run`, `eat`, `slap`)
- `CCONJ`: Coordinating Conjunctions (e.g. `but`, `and`, `yet`)

For a more in-depth explanation on the above part-of-speech tags, visit [Universal POS Tags](https://universaldependencies.org/u/pos/all.html).

In [7]:
class Token(enum.Enum):
    """
    Some simple tokens. Very inconclusive, but simple.
    """

    ADJ = 'Adjective'
    ADV = 'Adverb'
    NOUN = 'Noun'
    PUNCT = 'Punctuation'
    VERB = 'Verb'
    CCONJ = 'Coordinating Conjunction'